<a href="https://colab.research.google.com/github/MatteusStranger/final_project_ia/blob/master/Captcha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.kaggle.com/neurol/keras-classifier
# https://www.kaggle.com/shawon10/captcha-recognition
# https://www.kaggle.com/dhruvaditya/captcha-recognition

In [2]:
!gdown --id "12972ui35ROKbPqDDDuMG47Sdhl-VuVJq"
!unzip -q d1

Downloading...
From: https://drive.google.com/uc?id=12972ui35ROKbPqDDDuMG47Sdhl-VuVJq
To: /content/d1.zip
893MB [00:06, 142MB/s]
replace dataset/svMiN.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A


In [3]:
#Imports

from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from pathlib import Path

Using TensorFlow backend.


In [4]:
def create_net(num_symbols):

    # Camada de entrada
    img = layers.Input(shape=img_shape) 
    
    # Segunda camada - convolução com relu, podemos experimentar com sigma
    conv1 = layers.Conv2D(16, (3, 3), padding='valid', activation='relu', use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros')(img)    
    mp1 = layers.MaxPooling2D(padding='same')(conv1)
    
    # Terceira camada - convolução com relu, podemos experimentar com sigma
    conv2 = layers.Conv2D(32, (3, 3), padding='valid', activation='relu', use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)
    
    # Quarta camada - convolução com relu, podemos experimentar com sigma
    conv3 = layers.Conv2D(32, (3, 3), padding='valid', activation='relu', use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros')(mp2)
    bn = layers.BatchNormalization()(conv3)
    
    # Camada de saída
    mp3 = layers.MaxPooling2D(padding='same')(bn) 
    flat = layers.Flatten()(mp3)
    
    outs = []
    for i in range(5):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, name='digit%d' % i, activation='sigmoid')(drop)

        outs.append(res)
    
    model = Model(img, outs)
    model.compile('adam', loss=['categorical_crossentropy', 'categorical_crossentropy',
                                   'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
    model.summary()
    return model

In [5]:
def read_dataset_batch(symbols, batch_files, captcha_dir = 'dataset'):
    
    num_symbols = len(symbols)
    n_samples = len(batch_files)
    #X = np.zeros((n_samples, 50, 200, 1))
    #X = np.zeros((n_samples, 60, 160, 1))
    X = np.zeros((n_samples, 30, 80, 1))
    y = np.zeros((5, n_samples, num_symbols))

    for i in range(n_samples):
        
        pic = batch_files[i];
        img = cv2.imread(os.path.join(captcha_dir, pic), cv2.IMREAD_GRAYSCALE)
        #img = cv2.imread(pic, cv2.IMREAD_GRAYSCALE)
        #print(captcha_dir);
        pic_target = pic[:-4] # retira o .png
        #print(pic_target);
        #print(img);
        # Se nome do arquivo da imagem tem até 5 caracteres
        if len(pic_target) < 6:
            
            # Coloca o valor dos pixels entre 0 e 1
            img = img / 255.
            #print(img.shape)
            img = cv2.resize(img, (int(160/2), int(60/2)), interpolation=cv2.INTER_AREA)
            #print(img.shape)
            img = np.reshape(img, (img.shape[0], img.shape[1], 1))
            #img = np.reshape(img, (30, 80, 1))
            # cada captcha possui 5 caracteres
            targs = np.zeros((5, num_symbols))
            
            # Seta a sequência de letras alvo
            # O Alvo será uma matriz de todos símbolos pelas 5 posições presentes no captcha.
            # Se aquela posição corresponde aquele símbolo o valor da célula será 1, e 0 caso contrário.
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
                
            X[i] = img
            y[:, i] = targs
            
            if(i%1000 == 0):
                print(str(i) + " imagens read");

    return X, y

In [6]:

def create_net2(g_learning_rate=0.1,g_beta_1=0.1):
    # 50 200 1
    img = layers.Input(shape=img_shape)
    c1 = layers.Conv2D(48, kernel_size=5, strides=1, padding='same',name="c1")(img) # 25 100 48 
    p1  = layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid',name="p1")(c1)
    c2 = layers.Conv2D(48, kernel_size=5, strides=1, padding='same',name="c2")(p1) # 25 100 48 
    p2 = layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid',name="p2")(c2) # 12 50 48
    c3 = layers.Conv2D(64, kernel_size=5, strides=1, padding='same',name="c3")(p2) # 25 100 64
    p3 = layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid',name="p3")(c3) # 6 25 64
    flatten = layers.Flatten(name="flatten")(p3)
    outs = []
    l1 = layers.Dense(512, activation='relu')(flatten)
    for _ in range(5):
        res = layers.Dense(64, activation='softmax')(l1)
        outs.append(res)
    model = Model(img, outs)
    #opt = optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss=['categorical_crossentropy', 'categorical_crossentropy',
                                   'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'])
    #model.compile(optimizer=Adam(lr=g_learning_rate, beta_1=g_beta_1), loss=['categorical_crossentropy', 'categorical_crossentropy',
    #                               'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'])
    model.summary()
    return model


In [7]:
def read_name_files(files_dir = 'dataset'):

    folders = []

    # r=root, d=directories, f = files
    for r, d, f in os.walk(files_dir):
        # Nada
        s = "nada";
    return f

In [8]:
files = read_name_files()
print(len(files))

99994


In [9]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [10]:
def concat_history(history):
    
    global digit_acc, val_digit_acc, digit_loss, val_digit_loss, loss, val_loss
    
    digit_acc_now = [history.history['digit%d_accuracy' % i] for i in range(5)];
    val_digit_acc_now = [history.history['val_digit%d_accuracy' % i] for i in range(5)];
    
    digit_loss_now = [history.history['digit%d_loss' % i] for i in range(5)];
    val_digit_loss_now = [history.history['val_digit%d_loss' % i] for i in range(5)];
    
    loss_now = history.history['loss'];
    val_loss_now = history.history['val_loss'];    
    
    for i in range(5):
        digit_acc[i].extend(digit_acc_now[i])
        val_digit_acc[i].extend(val_digit_acc_now[i])
        
        digit_loss[i].extend(digit_loss_now[i])
        val_digit_loss[i].extend(val_digit_loss_now[i])
        
    loss.extend(loss_now)
    val_loss.extend(val_loss_now)

In [11]:



# checkpoint

outputFolder = './output-captcha'
Path(outputFolder).mkdir(exist_ok=True)
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)
filepath=outputFolder+"/weights-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, \
                             save_best_only=False, save_weights_only=True, \
                             mode='auto')
callbacks_list = [checkpoint]

In [12]:
symbols = string.ascii_lowercase + string.ascii_uppercase + "0123456789"
#img_shape = (60, 160, 1)
img_shape = (30, 80, 1)
net = create_net(len(symbols))

# Estatísticas
digit_acc = [[] for _ in range(5)]
val_digit_acc = [[] for _ in range(5)]

digit_loss = [[] for _ in range(5)]
val_digit_loss = [[] for _ in range(5)]

loss = []
val_loss = []

i = 0

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 80, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 78, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 14, 39, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 12, 37, 32)   4640        max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [18]:
for batch_files in batch(files, 2):    
    X_train, y_train = read_dataset_batch(symbols, batch_files)
    #print(X_train.shape())
    print("Inicio Treino #", str(i));
    i = i + 1
    history = net.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], epochs=50, validation_split=0.2, batch_size=300, callbacks=callbacks_list,verbose=0)
    concat_history(history);

0 imagens read
Inicio Treino # 7

Epoch 00001: saving model to ./output-captcha/weights-01-10.85.hdf5

Epoch 00002: saving model to ./output-captcha/weights-02-6.92.hdf5

Epoch 00003: saving model to ./output-captcha/weights-03-5.01.hdf5

Epoch 00004: saving model to ./output-captcha/weights-04-1.75.hdf5

Epoch 00005: saving model to ./output-captcha/weights-05-1.72.hdf5

Epoch 00006: saving model to ./output-captcha/weights-06-2.07.hdf5

Epoch 00007: saving model to ./output-captcha/weights-07-0.51.hdf5

Epoch 00008: saving model to ./output-captcha/weights-08-0.04.hdf5

Epoch 00009: saving model to ./output-captcha/weights-09-0.01.hdf5

Epoch 00010: saving model to ./output-captcha/weights-10-0.82.hdf5

Epoch 00011: saving model to ./output-captcha/weights-11-0.00.hdf5

Epoch 00012: saving model to ./output-captcha/weights-12-1.70.hdf5

Epoch 00013: saving model to ./output-captcha/weights-13-0.01.hdf5

Epoch 00014: saving model to ./output-captcha/weights-14-0.00.hdf5

Epoch 00015: 

KeyboardInterrupt: ignored

In [ ]:

def predict(img, n_symbols):
    
    #img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) / 255.
    res = np.array(net.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, n_symbols))
    l_ind = []
    probs = []
    
    for a in ans:
        l_ind.append(np.argmax(a))
        probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt, sum(probs) / 5

In [ ]:
    
    
    global digit_acc, val_digit_acc, loss, val_loss
    acc = 0;
    val_acc = 0;
    
    
    # Valores por digito de erro e acurácia
    for i in range(5):
        plt.plot(digit_acc[i], label='digit%d_accuracy' % i)
        plt.plot(val_digit_acc[i], label='val_digit%d_accuracy' % i)
        #print (digit_acc[i]);
        #acc = acc + digit_acc[i];
        #val_acc = val_acc + val_digit_acc[i];
        
    plt.title('Model Digit Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()
    
    #for i in range(5):
    #    plt.plot(digit_loss[i], label='digit%d_loss' % i)
    #    plt.plot(val_digit_loss[i], label='val_digit%d_loss' % i)        
        
    #plt.title('Model Digit Loss')
    #plt.ylabel('Accuracy')
    #plt.xlabel('Epoch')
    #plt.legend()
    #plt.show()

    # Valores gerais de erro e acurácia
    plt.plot(loss)
    plt.plot(val_loss)
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
    
    #plt.plot(acc)
    #plt.plot(val_acc)
    #plt.title('Model Accuracy')
    #plt.ylabel('Accuracy')
    #plt.xlabel('Epoch')
    #plt.legend(['Train', 'Test'], loc='upper left')
    #plt.show()

In [ ]:
plot_diagram()

In [ ]:
# Leitura da pasta teste
n_dir = 'images/dataset_test';
files = read_name_files(files_dir = n_dir)
X_test, y_test = read_dataset_batch(symbols, files, captcha_dir = n_dir)
net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
        
    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = 10.
    #for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    #    plt.text(j, i, format(cm[i, j], fmt),
    #             horizontalalignment="center",
    #             color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:

# 7i4iB
def accuracy(x, y, model):
    
    # Estatísticas
    # index 5 = matriz total, de todos os digitos.
    confusion_digit_predict = [[] for _ in range(6)]
    confusion_digit_true = [[] for _ in range(6)]


    y_d1 = np.argmax(y[0], axis=1);
    y_d2 = np.argmax(y[1], axis=1);
    y_d3 = np.argmax(y[2], axis=1);
    y_d4 = np.argmax(y[3], axis=1);
    y_d5 = np.argmax(y[4], axis=1);
    
    d1_num_correct = 0
    d2_num_correct = 0
    d3_num_correct = 0
    d4_num_correct = 0
    d5_num_correct = 0
    
    for i in range(x.shape[0]):
        capt, prop = predict(x[i], len(symbols))        
        d1_num_correct = d1_num_correct + np.sum(capt[0] == symbols[y_d1[i]])
        d2_num_correct = d2_num_correct + np.sum(capt[1] == symbols[y_d2[i]])
        d3_num_correct = d3_num_correct + np.sum(capt[2] == symbols[y_d3[i]])
        d4_num_correct = d4_num_correct + np.sum(capt[3] == symbols[y_d4[i]])
        d5_num_correct = d5_num_correct + np.sum(capt[4] == symbols[y_d5[i]])
        
        confusion_digit_predict[0].extend(capt[0]);
        confusion_digit_true[0].extend(symbols[y_d1[i]]);
        
        confusion_digit_predict[1].extend(capt[1]);
        confusion_digit_true[1].extend(symbols[y_d2[i]]);
        
        confusion_digit_predict[2].extend(capt[2]);
        confusion_digit_true[2].extend(symbols[y_d3[i]]);
        
        confusion_digit_predict[3].extend(capt[3]);
        confusion_digit_true[3].extend(symbols[y_d4[i]]);
        
        confusion_digit_predict[4].extend(capt[4]);
        confusion_digit_true[4].extend(symbols[y_d5[i]]);
        
        confusion_digit_predict[5].extend(capt[0]);
        confusion_digit_true[5].extend(symbols[y_d1[i]]);
        confusion_digit_predict[5].extend(capt[1]);
        confusion_digit_true[5].extend(symbols[y_d2[i]]);
        confusion_digit_predict[5].extend(capt[2]);
        confusion_digit_true[5].extend(symbols[y_d3[i]]);
        confusion_digit_predict[5].extend(capt[3]);
        confusion_digit_true[5].extend(symbols[y_d4[i]]);
        confusion_digit_predict[5].extend(capt[4]);
        confusion_digit_true[5].extend(symbols[y_d5[i]]);
        
    
    # Plota matizes de confusion:
    #print(confusion_matrix(confusion_digit_predict[0], confusion_digit_true[0]))
    plot_confusion_matrix(confusion_matrix(confusion_digit_predict[0], confusion_digit_true[0]), symbols, title='Confusion matrix D1')
    plot_confusion_matrix(confusion_matrix(confusion_digit_predict[1], confusion_digit_true[1]), symbols, title='Confusion matrix D2')
    plot_confusion_matrix(confusion_matrix(confusion_digit_predict[2], confusion_digit_true[2]), symbols, title='Confusion matrix D3')
    plot_confusion_matrix(confusion_matrix(confusion_digit_predict[3], confusion_digit_true[3]), symbols, title='Confusion matrix D4')
    plot_confusion_matrix(confusion_matrix(confusion_digit_predict[4], confusion_digit_true[4]), symbols, title='Confusion matrix D5')
    
    plot_confusion_matrix(confusion_matrix(confusion_digit_predict[5], confusion_digit_true[5]), symbols, title='Confusion matrix Geral')
    
        
    
    num_samples = x.shape[0];
    num_caracteres = len(capt)*num_samples;
    num_correct = d1_num_correct + d2_num_correct + d3_num_correct + d4_num_correct + d5_num_correct
    
    d1_accuracy = float(d1_num_correct)/num_samples
    d2_accuracy = float(d2_num_correct)/num_samples
    d3_accuracy = float(d3_num_correct)/num_samples
    d4_accuracy = float(d4_num_correct)/num_samples
    d5_accuracy = float(d5_num_correct)/num_samples
    accuracy = float(num_correct)/num_caracteres
    
    #plot_confusion_matrix(confusion_matrix(true_class, predicted_class), className)
    
    return accuracy, d1_accuracy, d2_accuracy, d3_accuracy, d4_accuracy, d5_accuracy

In [ ]:
print ("Accuracy on test data is: %0.2f, acc_d1= %0.2f, acc_d2= %0.2f, acc_d3= %0.2f, acc_d4= %0.2f, acc_d5= %0.2f"%accuracy(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]], net))

In [ ]:
j = 1
for i in (1, 3, 5):
    print("--- Amostra #" + str(j) + " ---")    
    fig = plt.figure()
    img = X_train[i,:,:]
    img.shape = (30,80)    
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()
    capt, prop = predict(img, len(symbols))
    print("Predict: " + capt)
    print("\n")
    
#net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])

In [ ]:
j = 1
for i in (0, 3, 5):
    print("--- Amostra #" + str(j) + " ---")    
    fig = plt.figure()
    img = X_test[i,:,:]
    img.shape = (30,80)    
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()
    capt, prop = predict(img, len(symbols))
    print("Predict: " + capt)
    print("\n")